In [7]:
"""
Optimized vegetable-classification training script (PyTorch)
Improvements: label smoothing, AdamW, CosineAnnealingLR, EMA, TTA, AMP, stronger augmentation
Author: For Abdullah (teacher style) - Perfected Version
"""

import os
import random
import time
from pathlib import Path
from collections import OrderedDict

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
from PIL import Image
import kagglehub

# -----------------------
# CONFIG
# -----------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Download dataset
print("Downloading dataset...")
path = kagglehub.dataset_download("misrakahmed/vegetable-image-dataset")
print("Path to dataset files:", path)

# User parameters - change these as needed
DATA_ROOT = path

# Better directory handling - check multiple possible structures
possible_structures = [
    ("Vegetable Images/train", "Vegetable Images/validation", "Vegetable Images/test"),
    ("train", "validation", "test"),
]

TRAIN_DIR = VAL_DIR = TEST_DIR = None
for train_sub, val_sub, test_sub in possible_structures:
    train_path = os.path.join(DATA_ROOT, train_sub)
    val_path = os.path.join(DATA_ROOT, val_sub)
    test_path = os.path.join(DATA_ROOT, test_sub)
    if all(os.path.exists(p) for p in [train_path, val_path, test_path]):
        TRAIN_DIR, VAL_DIR, TEST_DIR = train_path, val_path, test_path
        print(f"Found dataset structure: {train_sub}")
        break

if TRAIN_DIR is None:
    raise ValueError(f"Dataset directories not found. Check structure at {DATA_ROOT}")

BATCH_SIZE = 32
IMAGE_SIZE = 224
EPOCHS = 20
LR = 3e-4
WEIGHT_DECAY = 1e-2
NUM_WORKERS = 4
NUM_CLASSES = None  # will be inferred
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = True       # Automatic Mixed Precision (faster & less memory)
USE_EMA = True       # Use Exponential Moving Average for final model
EMA_DECAY = 0.9997
USE_TTA = True       # Use test-time augmentation at inference
TTA_TRANSFORMS = 5   # number of augmented views per test image

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------
# HELPERS: seed + device info
# -----------------------
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

print("Device:", DEVICE)

# -----------------------
# TRANSFORMS (train/val/test)
# -----------------------
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.1),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05)
    ], p=0.7),
    transforms.RandomGrayscale(p=0.03),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Validation/test transforms: deterministic
val_transform = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.15)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# -----------------------
# DATASET LOADING
# -----------------------
train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
val_ds = datasets.ImageFolder(VAL_DIR, transform=val_transform)
test_ds = datasets.ImageFolder(TEST_DIR, transform=val_transform)

class_names = train_ds.classes
NUM_CLASSES = len(class_names)
print(f"Classes ({NUM_CLASSES}): {class_names}")
print(f"Train samples: {len(train_ds)}, Val samples: {len(val_ds)}, Test samples: {len(test_ds)}")

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, 
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=NUM_WORKERS, pin_memory=True)

# -----------------------
# MODEL: pretrained convnext_tiny (preferred) or resnet18 fallback
# -----------------------
def create_model(num_classes=NUM_CLASSES):
    try:
        # prefer convnext if torchvision version supports it
        model = models.convnext_tiny(weights='IMAGENET1K_V1')
        # replace classifier
        in_f = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_f, num_classes)
        print("Using torchvision.convnext_tiny")
    except Exception:
        print("convnext not available, using resnet18")
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        in_f = model.fc.in_features
        model.fc = nn.Linear(in_f, num_classes)
    return model

model = create_model().to(DEVICE)

# -----------------------
# EMA (Exponential Moving Average) helper
# -----------------------
class ModelEMA:
    """ Maintains exponential moving average of model parameters """
    def __init__(self, model, decay=EMA_DECAY):
        self.ema = OrderedDict()
        self.decay = decay
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.ema[name] = param.detach().clone().to('cpu')

    def update(self, model):
        for name, param in model.named_parameters():
            if name in self.ema and param.requires_grad:
                new_v = param.detach().cpu()
                self.ema[name] = (1.0 - self.decay) * new_v + self.decay * self.ema[name]

    def apply_shadow(self, model):
        # Save current params, then load ema params into model
        self.backup = {}
        for name, param in model.named_parameters():
            if name in self.ema:
                self.backup[name] = param.detach().clone()
                param.data.copy_(self.ema[name].to(param.device))

    def restore(self, model):
        for name, param in model.named_parameters():
            if name in self.backup:
                param.data.copy_(self.backup[name].to(param.device))
        self.backup = {}

ema = ModelEMA(model) if USE_EMA else None

# -----------------------
# LOSS, OPTIMIZER, SCHEDULER
# -----------------------
# use label smoothing inside CrossEntropyLoss (PyTorch >=1.10 supports label_smoothing)
criterion = nn.CrossEntropyLoss(label_smoothing=0.05).to(DEVICE)

# AdamW optimizer recommended for modern training
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# Cosine annealing scheduler for smooth decays
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# -----------------------
# TRAIN / VALID functions (with AMP & gradient clipping)
# -----------------------
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)

def train_one_epoch(model, loader, criterion, optimizer, device, epoch, ema_obj=None, max_grad_norm=1.0):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pbar = tqdm(loader, desc=f"Train E{epoch}")
    for images, labels in pbar:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=USE_AMP):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        # gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        scaler.step(optimizer)
        scaler.update()

        # EMA update
        if ema_obj is not None:
            ema_obj.update(model)

        preds = outputs.argmax(dim=1)
        running_loss += loss.item() * images.size(0)
        correct += (preds == labels).sum().item()
        total += images.size(0)
        pbar.set_postfix(loss=f"{running_loss/total:.4f}", acc=f"{correct/total:.4f}")

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Eval"):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            preds = outputs.argmax(dim=1)
            running_loss += loss.item() * images.size(0)
            correct += (preds == labels).sum().item()
            total += images.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
    return running_loss/total, correct/total, np.array(all_targets), np.array(all_preds)

# -----------------------
# TTA inference (properly seeded for reproducibility)
# -----------------------
def tta_evaluate(model, dataset, device, n_tta=TTA_TRANSFORMS):
    """
    Test-Time Augmentation with proper seeding for reproducibility
    """
    model.eval()
    tta_preds = []
    tta_targets = []
    
    print(f"Running TTA inference with {n_tta} augmentations per image...")
    
    for idx in tqdm(range(len(dataset))):
        img_path, label = dataset.samples[idx]
        pil_image = Image.open(img_path).convert("RGB")
        
        probs = []
        for tta_idx in range(n_tta):
            # Set seed for reproducible TTA
            seed = SEED + idx * n_tta + tta_idx
            random.seed(seed)
            torch.manual_seed(seed)
            np.random.seed(seed)
            
            # Create augmented transform
            tta_transform = transforms.Compose([
                transforms.Resize(int(IMAGE_SIZE * 1.1)),
                transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.9, 1.0)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            
            img_t = tta_transform(pil_image).unsqueeze(0).to(device)
            with torch.no_grad():
                out = torch.softmax(model(img_t), dim=1)
            probs.append(out.cpu().numpy())
        
        # Average probabilities across augmentations
        avg_prob = np.mean(np.vstack(probs), axis=0)
        pred = np.argmax(avg_prob)
        tta_preds.append(pred)
        tta_targets.append(label)
    
    # Reset seed
    set_seed(SEED)
    
    return np.array(tta_targets), np.array(tta_preds)

# -----------------------
# CHECKPOINT helper
# -----------------------
def save_checkpoint(state, filename):
    torch.save(state, filename)
    print(f"Checkpoint saved: {filename}")

# -----------------------
# TRAINING LOOP
# -----------------------
best_val_acc = 0.0
best_ckpt_path = OUT_DIR / "best_model.pth"
best_ckpt_ema_path = OUT_DIR / "best_model_ema.pth"

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

for epoch in range(1, EPOCHS + 1):
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, 
                                           DEVICE, epoch, ema_obj=ema)
    # Step scheduler after training epoch
    scheduler.step()
    
    # validate with normal weights
    val_loss, val_acc, _, _ = evaluate(model, val_loader, criterion, DEVICE)

    # If using EMA: evaluate EMA-shadow as well (recommended)
    if ema is not None:
        ema.apply_shadow(model)
        val_loss_ema, val_acc_ema, _, _ = evaluate(model, val_loader, criterion, DEVICE)
        ema.restore(model)
    else:
        val_loss_ema, val_acc_ema = val_loss, val_acc

    elapsed = time.time() - t0
    print(f"\nEpoch {epoch}/{EPOCHS} - time: {elapsed:.1f}s")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
    print(f"  Val Loss:   {val_loss:.4f}, Val Acc:   {val_acc*100:.2f}%")
    if ema is not None:
        print(f"  Val EMA Acc: {val_acc_ema*100:.2f}%")

    # choose checkpoint decision based on EMA val acc if EMA is used
    current_val_metric = val_acc_ema if ema is not None else val_acc

    if current_val_metric > best_val_acc:
        best_val_acc = current_val_metric
        # Save normal model
        save_checkpoint({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "val_acc": val_acc,
            "class_names": class_names,
        }, best_ckpt_path)
        
        if ema is not None:
            # Save EMA params into separate checkpoint file
            ema.apply_shadow(model)
            save_checkpoint({
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "val_acc": val_acc_ema,
                "class_names": class_names,
            }, best_ckpt_ema_path)
            ema.restore(model)
        print(f"  ✓ New best model (val_acc={best_val_acc:.4f})")

print("\n" + "="*60)
print(f"TRAINING COMPLETE. Best val acc: {best_val_acc:.4f}")
print("="*60)

# -----------------------
# FINAL EVALUATION ON TEST SET
# -----------------------
print("\n" + "="*60)
print("FINAL TEST EVALUATION")
print("="*60)

# prefer EMA checkpoint if exists
if os.path.exists(best_ckpt_ema_path):
    ckpt = torch.load(best_ckpt_ema_path, map_location=DEVICE)
    model.load_state_dict(ckpt["model_state"])
    print("Loaded EMA best checkpoint for final test evaluation.")
else:
    ckpt = torch.load(best_ckpt_path, map_location=DEVICE)
    model.load_state_dict(ckpt["model_state"])
    print("Loaded standard best checkpoint for final test evaluation.")

# Normal forward test evaluation
test_loss, test_acc, test_targets, test_preds = evaluate(model, test_loader, criterion, DEVICE)
print(f"\nStandard Test Results:")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Acc:  {test_acc*100:.4f}%")

# Classification report + confusion matrix
print("\nClassification Report:")
print(classification_report(test_targets, test_preds, target_names=class_names, digits=4))
print("\nConfusion Matrix:")
print(cm := confusion_matrix(test_targets, test_preds))

# -----------------------
# OPTIONAL: TTA-based evaluation (if USE_TTA True)
# -----------------------
if USE_TTA:
    print("\n" + "="*60)
    print("TEST-TIME AUGMENTATION EVALUATION")
    print("="*60)
    
    tta_targets, tta_preds = tta_evaluate(model, test_ds, DEVICE, n_tta=TTA_TRANSFORMS)
    tta_acc = accuracy_score(tta_targets, tta_preds)
    
    print(f"\nTTA Test Results (with {TTA_TRANSFORMS} augmentations):")
    print(f"  TTA Accuracy: {tta_acc*100:.4f}%")
    print(f"  Improvement:  {(tta_acc - test_acc)*100:+.4f}%")
    
    print("\nTTA Classification Report:")
    print(classification_report(tta_targets, tta_preds, target_names=class_names, digits=4))

print("\n" + "="*60)
print("ALL DONE! ✓")
print("="*60)

Path to dataset files: /kaggle/input/vegetable-image-dataset
Found dataset structure: Vegetable Images/train
Device: cuda
Classes (15): ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']
Train samples: 15000, Val samples: 3000, Test samples: 3000


Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 207MB/s] 


Using torchvision.convnext_tiny


/tmp/ipykernel_47/2815081326.py:203: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)



STARTING TRAINING


...


Epoch 20/20 - time: 486.2s
  Train Loss: 0.3119, Train Acc: 99.99%
  Val Loss:   0.3118, Val Acc:   100.00%
  Val EMA Acc: 100.00%

TRAINING COMPLETE. Best val acc: 1.0000

FINAL TEST EVALUATION
Loaded EMA best checkpoint for final test evaluation.


Eval: 100%|██████████| 94/94 [00:09<00:00, 10.19it/s]



Standard Test Results:
  Test Loss: 0.5115
  Test Acc:  99.8667%

Classification Report:
              precision    recall  f1-score   support

        Bean     1.0000    1.0000    1.0000       200
Bitter_Gourd     1.0000    0.9900    0.9950       200
Bottle_Gourd     1.0000    0.9950    0.9975       200
     Brinjal     0.9950    1.0000    0.9975       200
    Broccoli     1.0000    1.0000    1.0000       200
     Cabbage     1.0000    0.9950    0.9975       200
    Capsicum     1.0000    1.0000    1.0000       200
      Carrot     1.0000    1.0000    1.0000       200
 Cauliflower     0.9950    1.0000    0.9975       200
    Cucumber     1.0000    1.0000    1.0000       200
      Papaya     0.9950    1.0000    0.9975       200
      Potato     1.0000    1.0000    1.0000       200
     Pumpkin     0.9950    1.0000    0.9975       200
      Radish     1.0000    1.0000    1.0000       200
      Tomato     1.0000    1.0000    1.0000       200

    accuracy                         0.9987 

100%|██████████| 3000/3000 [02:44<00:00, 18.21it/s]


TTA Test Results (with 5 augmentations):
  TTA Accuracy: 99.9000%
  Improvement:  +0.0333%

TTA Classification Report:
              precision    recall  f1-score   support

        Bean     1.0000    1.0000    1.0000       200
Bitter_Gourd     1.0000    0.9950    0.9975       200
Bottle_Gourd     1.0000    0.9950    0.9975       200
     Brinjal     1.0000    1.0000    1.0000       200
    Broccoli     1.0000    1.0000    1.0000       200
     Cabbage     1.0000    0.9950    0.9975       200
    Capsicum     1.0000    1.0000    1.0000       200
      Carrot     1.0000    1.0000    1.0000       200
 Cauliflower     0.9950    1.0000    0.9975       200
    Cucumber     1.0000    1.0000    1.0000       200
      Papaya     0.9950    1.0000    0.9975       200
      Potato     1.0000    1.0000    1.0000       200
     Pumpkin     0.9950    1.0000    0.9975       200
      Radish     1.0000    1.0000    1.0000       200
      Tomato     1.0000    1.0000    1.0000       200

    accuracy  